In [1]:
#Optimizacion Bayesiana de hiperparametros de  rpart
#funciona automaticamente con EXPERIMENTOS
#va generando incrementalmente salidas para kaggle

#limpio la memoria
rm( list=ls() )  #remove all objects
gc()             #garbage collection

library("data.table")
library("rlist")
library("yaml")

library("rpart")
library("parallel")

#paquetes necesarios para la Bayesian Optimization
library("DiceKriging")
library("mlrMBO")

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,492078,26.3,1085047,58,641780,34.3
Vcells,940890,7.2,8388608,64,1753993,13.4


Warning message:
“package ‘rlist’ was built under R version 3.6.3”
Warning message:
“package ‘yaml’ was built under R version 3.6.3”
Warning message:
“package ‘rpart’ was built under R version 3.6.3”
Warning message:
“package ‘DiceKriging’ was built under R version 3.6.3”
Warning message:
“package ‘mlrMBO’ was built under R version 3.6.3”
Loading required package: mlr

Warning message:
“package ‘mlr’ was built under R version 3.6.3”
Loading required package: ParamHelpers

Warning message:
“package ‘ParamHelpers’ was built under R version 3.6.3”
Warning message: 'mlr' is in 'maintenance-only' mode since July 2019.
Future development will only happen in 'mlr3'
(<https://mlr3.mlr-org.com>). Due to the focus on 'mlr3' there might be
uncaught bugs meanwhile in {mlr} - please consider switching.

Loading required package: smoof

Warning message:
“package ‘smoof’ was built under R version 3.6.3”
Loading required package: checkmate

Warning message:
“package ‘checkmate’ was built under R versi

In [2]:
#para poder usarlo en la PC y en la nube
switch ( Sys.info()[['sysname']],
         Windows = { directory.root   <-  "M:\\" },   #Microsoft Windows
         Darwin  = { directory.root   <-  "~/dm/" },  #Apple MAC
         Linux   = { directory.root   <-  "~/buckets/b1/crudo/" }  #Entorno Google Cloud
       )
#defino la carpeta donde trabajo
setwd( directory.root )

In [3]:
kexperimento  <- NA   #NA si se corre la primera vez, un valor concreto si es para continuar procesando

kscript           <- "03.01 - BO Rpart"
karch_generacion  <- "./datasetsOri/paquete_premium_202011.csv"
karch_aplicacion  <- "./datasetsOri/paquete_premium_202101.csv"
kBO_iter    <-  200   #cantidad de iteraciones de la Optimizacion Bayesiana

hs  <- makeParamSet(
          makeNumericParam("cp"       , lower= -1   , upper=    0.1),
          makeIntegerParam("minsplit" , lower=  1L  , upper= 8000L),  #la letra L al final significa ENTERO
          makeIntegerParam("minbucket", lower=  1L  , upper= 2000L),
          makeIntegerParam("maxdepth" , lower=  3L  , upper=   20L),
          forbidden = quote( minbucket > 0.5*minsplit ) )


ksemilla_azar  <- 102191

In [4]:
#------------------------------------------------------------------------------
#Funcion que lleva el registro de los experimentos

get_experimento  <- function()
{
  if( !file.exists( "./maestro.yaml" ) )  cat( file="./maestro.yaml", "experimento: 1000" )

  exp  <- read_yaml( "./maestro.yaml" )
  experimento_actual  <- exp$experimento

  exp$experimento  <- as.integer(exp$experimento + 1)
  Sys.chmod( "./maestro.yaml", mode = "0644", use_umask = TRUE)
  write_yaml( exp, "./maestro.yaml" )
  Sys.chmod( "./maestro.yaml", mode = "0444", use_umask = TRUE) #dejo el archivo readonly

  return( experimento_actual )
}

In [5]:
#------------------------------------------------------------------------------
#graba a un archivo los componentes de lista
#para el primer registro, escribe antes los titulos

loguear  <- function( reg, arch=NA, folder="./work/", ext=".txt", verbose=TRUE )
{
  archivo  <- arch
  if( is.na(arch) )  archivo  <- paste0(  folder, substitute( reg), ext )

  if( !file.exists( archivo ) )  #Escribo los titulos
  {
    linea  <- paste0( "fecha\t", 
                      paste( list.names(reg), collapse="\t" ), "\n" )

    cat( linea, file=archivo )
  }

  linea  <- paste0( format(Sys.time(), "%Y%m%d %H%M%S"),  "\t",     #la fecha y hora
                    gsub( ", ", "\t", toString( reg ) ),  "\n" )

  cat( linea, file=archivo, append=TRUE )  #grabo al archivo

  if( verbose )  cat( linea )   #imprimo por pantalla
}

In [6]:
#------------------------------------------------------------------------------
#funcion para particionar, es la que Andres reemplaza con caret

particionar  <- function( data, division, agrupa="", campo="fold", start=1, seed=NA )
{
  if( !is.na( seed)  )   set.seed( seed )

  bloque  <- unlist( mapply(  function(x,y) { rep( y, x ) }, division, seq( from=start, length.out=length(division) )  ) )

  data[ , (campo) :=  sample( rep( bloque, ceiling(.N/length(bloque))) )[1:.N],
           by= agrupa ]
}
#------------------------------------------------------------------------------

In [7]:
ArbolSimple  <- function( fold_test, data, param )
{
  #genero el modelo
  modelo  <- rpart("clase_ternaria ~ .", 
                   data= data[ fold != fold_test, ],
                   xval= 0,
                   control= param )

  #aplico el modelo a los datos de testing, fold==2
  prediccion  <- predict( modelo, data[ fold==fold_test, ], type = "prob")

  prob_baja2  <- prediccion[, "BAJA+2"]

  ganancia_testing  <- sum(  data[ fold==fold_test ][ prob_baja2 >0.025,  ifelse( clase_ternaria=="BAJA+2", 48750, -1250 ) ] )

  return( ganancia_testing )
}

In [8]:
#------------------------------------------------------------------------------

ArbolesCrossValidation  <- function( data, param, qfolds, pagrupa, semilla )
{
  divi  <- rep( 1, qfolds )
  particionar( data, divi, seed=semilla, agrupa=pagrupa )

  ganancias  <- mcmapply( ArbolSimple, 
                          seq(qfolds), # 1 2 3 4 5  
                          MoreArgs= list( data, param), 
                          SIMPLIFY= FALSE,
                          mc.cores= 5 )   #se puede subir a 5 si posee Linux o Mac OS

  data[ , fold := NULL ]
  #devuelvo la primer ganancia y el promedio
  return( mean( unlist( ganancias )) *  qfolds )   #aqui normalizo la ganancia
}

In [9]:
#------------------------------------------------------------------------------
#esta funcion solo puede recibir los parametros que se estan optimizando
#el resto de los parametros se pasan como variables globales

EstimarGanancia  <- function( x )
{
   GLOBAL_iteracion  <<-  GLOBAL_iteracion + 1

   xval_folds  <- 5
   ganancia  <-  ArbolesCrossValidation( dataset, param=x, qfolds= xval_folds, pagrupa="clase_ternaria", semilla=ksemilla_azar )

   #si tengo una ganancia superadora, genero el archivo para Kaggle
   if(  ganancia > GLOBAL_ganancia_max )
   {
     GLOBAL_ganancia_max <<-  ganancia  #asigno la nueva maxima ganancia
    
     modelo  <- rpart("clase_ternaria ~ .",
                      data= dataset,
                      xval= 0,
                      control= x )

     #genero el vector con la prediccion, la probabilidad de ser positivo
     prediccion  <- predict( modelo, dapply)

     prob_baja2  <- prediccion[, "BAJA+2"]
     Predicted   <- ifelse( prob_baja2 > 0.025, 1, 0 )

     entrega  <-  as.data.table( list( "numero_de_cliente"=dapply$numero_de_cliente, "Predicted"=Predicted)  )

     #genero el archivo para Kaggle
     fwrite( entrega, 
             file= paste0(kkaggle, GLOBAL_iteracion, ".csv" ),
             sep=  "," )
   }

   #logueo 
   xx  <- x
   xx$xval_folds  <-  xval_folds
   xx$ganancia  <- ganancia
   loguear( xx,  arch= klog )


   return( ganancia )
}
#------------------------------------------------------------------------------

In [10]:
#Aqui empieza el programa

if( is.na(kexperimento ) )   kexperimento <- get_experimento()  #creo el experimento

In [11]:
#en estos archivos quedan los resultados
kbayesiana  <- paste0("./work/E",  kexperimento, "_rpart.RDATA" )
klog        <- paste0("./work/E",  kexperimento, "_rpart_log.txt" )
kkaggle     <- paste0("./kaggle/E",kexperimento, "_rpart_kaggle_" )

In [12]:
GLOBAL_ganancia_max  <-  -Inf
GLOBAL_iteracion  <- 0

In [13]:
if( file.exists(klog) )
{
 tabla_log  <- fread( klog)
 GLOBAL_iteracion  <- nrow( tabla_log ) -1
 GLOBAL_ganancia_max  <-  tabla_log[ , max(ganancia) ]
}

In [14]:
#cargo los datasets
dataset  <- fread(karch_generacion)   #donde entreno
dapply  <- fread(karch_aplicacion)    #donde aplico el modelo

In [15]:
#Aqui comienza la configuracion de la Bayesian Optimization

configureMlr( show.learner.output = FALSE)

funcion_optimizar  <- EstimarGanancia

#configuro la busqueda bayesiana,  los hiperparametros que se van a optimizar
#por favor, no desesperarse por lo complejo
obj.fun  <- makeSingleObjectiveFunction(
              fn=       funcion_optimizar,
              minimize= FALSE,   #estoy Maximizando la ganancia
              noisy=    TRUE,
              par.set=  hs,
              has.simple.signature = FALSE
             )

ctrl  <- makeMBOControl( save.on.disk.at.time= 600,  save.file.path= kbayesiana)
ctrl  <- setMBOControlTermination(ctrl, iters= kBO_iter )
ctrl  <- setMBOControlInfill(ctrl, crit= makeMBOInfillCritEI())

surr.km  <-  makeLearner("regr.km", predict.type= "se", covtype= "matern3_2", control= list(trace= TRUE))

In [ ]:
#inicio la optimizacion bayesiana
if(!file.exists(kbayesiana)) {
  run  <- mbo(obj.fun, learner = surr.km, control = ctrl)
} else  run  <- mboContinue( kbayesiana )   #retomo en caso que ya exista

Computing y column(s) for design. Not provided.



20210903 163127	-0.11803685661871	4037	499	8	5	8318750
20210903 163150	-0.807102801634755	6315	1462	6	5	7783750
20210903 163232	-0.72545432754996	4930	1064	10	5	8292500
20210903 163316	-0.2292730446934	3458	1326	14	5	7985000
20210903 163403	-0.435396663319261	7854	901	13	5	7873750
20210903 163457	-0.683192678492924	2173	868	7	5	8351250
20210903 163601	-0.344508100372332	5699	196	20	5	8343750
20210903 163626	-0.460279139662452	7118	68	5	5	7955000
20210903 163648	-0.253486779540253	6917	1641	4	5	7546250
20210903 163716	0.0116837216584829	5233	1956	18	5	0
20210903 163808	-0.955280557920923	3961	589	17	5	8306250
20210903 163847	-0.49549774767831	3917	1816	10	5	7812500
20210903 163908	-0.974445402333513	7773	319	4	5	7873750
20210903 163957	-0.113420343208127	5241	1005	15	5	8257500
20210903 164014	0.051272066764068	6795	249	4	5	0
20210903 164103	-0.0367552251787855	6506	544	15	5	8348750


[mbo] 0: cp=-0.118; minsplit=4037; minbucket=499; maxdepth=8 : y = 8.32e+06 : 60.1 secs : initdesign

[mbo] 0: cp=-0.807; minsplit=6315; minbucket=1462; maxdepth=6 : y = 7.78e+06 : 22.9 secs : initdesign

[mbo] 0: cp=-0.725; minsplit=4930; minbucket=1064; maxdepth=10 : y = 8.29e+06 : 42.0 secs : initdesign

[mbo] 0: cp=-0.229; minsplit=3458; minbucket=1326; maxdepth=14 : y = 7.98e+06 : 43.8 secs : initdesign

[mbo] 0: cp=-0.435; minsplit=7854; minbucket=901; maxdepth=13 : y = 7.87e+06 : 47.2 secs : initdesign

[mbo] 0: cp=-0.683; minsplit=2173; minbucket=868; maxdepth=7 : y = 8.35e+06 : 53.6 secs : initdesign

[mbo] 0: cp=-0.345; minsplit=5699; minbucket=196; maxdepth=20 : y = 8.34e+06 : 64.7 secs : initdesign

[mbo] 0: cp=-0.46; minsplit=7118; minbucket=68; maxdepth=5 : y = 7.96e+06 : 24.2 secs : initdesign

[mbo] 0: cp=-0.253; minsplit=6917; minbucket=1641; maxdepth=4 : y = 7.55e+06 : 22.9 secs : initdesign

[mbo] 0: cp=0.0117; minsplit=5233; minbucket=1956; maxdepth=18 : y = 0 : 27.

20210903 164155	-0.0644864398245086	7212	9	14	5	7668750


[mbo] 1: cp=-0.0645; minsplit=7212; minbucket=9; maxdepth=14 : y = 7.67e+06 : 49.9 secs : infill_ei



...

[mbo] 45: cp=-0.502; minsplit=2219; minbucket=1106; maxdepth=13 : y = 8.72e+06 : 43.9 secs : infill_ei



20210903 171456	-0.435813463565006	3280	618	7	5	8258750


[mbo] 46: cp=-0.436; minsplit=3280; minbucket=618; maxdepth=7 : y = 8.26e+06 : 36.0 secs : infill_ei



20210903 171531	-0.829333349400873	583	290	6	5	8672500


[mbo] 47: cp=-0.829; minsplit=583; minbucket=290; maxdepth=6 : y = 8.67e+06 : 31.2 secs : infill_ei



In [ ]:
quit( save="no" )